# Offline diagnostics for the CASTLE single output networks following Rasp et al. (2018) architecture

# Stats Computation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printe
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [3]:
module_path = os.path.abspath(os.path.join('..'))
# Relative imports
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
from utils.setup import SetupDiagnostics
from neural_networks.load_models import load_models, get_save_plot_folder, load_single_model
from neural_networks.model_diagnostics import ModelDiagnostics
from pathlib import Path
from utils.variable import Variable_Lev_Metadata
import pickle

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Load trained CASTLE models

In [6]:
argv  = ["-c", "../output_castle/training_28_custom_mirrored_functional/cfg_castle_training_run_1.yml"]
plot_dir = Path("../output_castle/training_28_custom_mirrored_functional/plots_offline_evaluation/run_1/stats/")
Path(plot_dir).mkdir(parents=True, exist_ok=True)

In [7]:
castle_setup = SetupDiagnostics(argv)

In [8]:
var_name = "prect"

### Load only models for one variables

In [10]:
castle_models = load_single_model(castle_setup, var_name)

KeyError: 'prec'

### Load all models

In [9]:
castle_models = load_models(castle_setup)


Load model: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/training_28_custom_mirrored_functional/models_castle/run_1/castleNN/r1.0-a1.0-b1.0-l1.0-mirrored/hl_256_256_256_256_256_256_256_256_256-act_ReLU-e_18/1_0_model.keras

Load model: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/training_28_custom_mirrored_functional/models_castle/run_1/castleNN/r1.0-a1.0-b1.0-l1.0-mirrored/hl_256_256_256_256_256_256_256_256_256-act_ReLU-e_18/1_1_model.keras

Load model: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/training_28_custom_mirrored_functional/models_castle/run_1/castleNN/r1.0-a1.0-b1.0-l1.0-mirrored/hl_256_256_256_256_256_256_256_256_256-act_ReLU-e_18/1_2_model.keras

Load model: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/training_28_custom_mirrored_functional/models_castle/run_1/castleNN/r1.0-a1.0-b1.0-l1.0-mirrored/hl_256_25

In [10]:
len(castle_models['castleNN'])

65

In [11]:
# Note: keys are variables not strings
castle_models['castleNN'].keys()

dict_keys(['tphystnd-3.64', 'tphystnd-7.59', 'tphystnd-14.36', 'tphystnd-24.61', 'tphystnd-38.27', 'tphystnd-54.6', 'tphystnd-72.01', 'tphystnd-87.82', 'tphystnd-103.32', 'tphystnd-121.55', 'tphystnd-142.99', 'tphystnd-168.23', 'tphystnd-197.91', 'tphystnd-232.83', 'tphystnd-273.91', 'tphystnd-322.24', 'tphystnd-379.1', 'tphystnd-445.99', 'tphystnd-524.69', 'tphystnd-609.78', 'tphystnd-691.39', 'tphystnd-763.4', 'tphystnd-820.86', 'tphystnd-859.53', 'tphystnd-887.02', 'tphystnd-912.64', 'tphystnd-936.2', 'tphystnd-957.49', 'tphystnd-976.33', 'tphystnd-992.56', 'phq-3.64', 'phq-7.59', 'phq-14.36', 'phq-24.61', 'phq-38.27', 'phq-54.6', 'phq-72.01', 'phq-87.82', 'phq-103.32', 'phq-121.55', 'phq-142.99', 'phq-168.23', 'phq-197.91', 'phq-232.83', 'phq-273.91', 'phq-322.24', 'phq-379.1', 'phq-445.99', 'phq-524.69', 'phq-609.78', 'phq-691.39', 'phq-763.4', 'phq-820.86', 'phq-859.53', 'phq-887.02', 'phq-912.64', 'phq-936.2', 'phq-957.49', 'phq-976.33', 'phq-992.56', 'fsnt', 'fsns', 'flnt', 'fl

## Compute Stats

In [12]:
# This variable does not exist in the code (but key nn_type is the same)
castle_model_type = "castleNN"
castle_setup.model_type = castle_model_type

In [13]:
castle_md = ModelDiagnostics(setup=castle_setup, 
                             models=castle_models[castle_model_type]) 

In [14]:
dict_keys = castle_models['castleNN'].keys()

### Single Variable

In [25]:
def save_single_stats():
    f_name = f"{i_time}_{n_time}_{var}.p"
    with open(os.path.join(plot_dir, f_name), "wb") as f:
        pickle.dump(castle_md.stats, f)

In [18]:
var = Variable_Lev_Metadata.parse_var_name(var_name)

In [21]:
i_time = "range" # this has to be range for it to work
n_time = 1440

castle_md.compute_stats(i_time, var, nTime=n_time)
castle_md.stats

Test batch size = 8192.
Time samples: 1440


{'hor_tsqmean': 1.9285970748918967e-14,
 'hor_tmean': 3.8232942259022e-08,
 'hor_mse': 1.3778499503189315e-14,
 'hor_tvar': 1.7824212875137258e-14,
 'hor_r2': 0.226978515140562}

In [26]:
save_single_stats()

In [27]:
def load_single_stats():
    f_name = f"{i_time}_{n_time}_{var}.p"
    with open(os.path.join(plot_dir, f_name), "rb") as f:
        var_stats = pickle.load(f)
    return var_stats

In [28]:
var_stats = load_single_stats()

In [29]:
var_stats

{'hor_tsqmean': 1.9285970748918967e-14,
 'hor_tmean': 3.8232942259022e-08,
 'hor_mse': 1.3778499503189315e-14,
 'hor_tvar': 1.7824212875137258e-14,
 'hor_r2': 0.226978515140562}

### All Variables

In [15]:
def get_save_str(idx_time, num_time=False):
    if type(idx_time) is int:
        idx_time_str = f"step-{idx_time}"
    elif type(idx_time) is str:
        if num_time:
            idx_time_str = f"{idx_time}-{num_time}"
        else:
            idx_time_str = f"{idx_time}-all"
    else:
        raise ValueError(f"Unkown value for idx_time: {idx_time}")
    
    return idx_time_str + ".p"
        

In [16]:
# Not function parameters, uses variables that are set in Notebook cells!!
def run_compute_stats():
    save_dir = Path(plot_dir, get_save_str(i_time, num_time=n_time))
    Path(save_dir).mkdir(parents=True, exist_ok=True)
                    
    stats_dict = dict()

    for var in dict_keys:
        print(f"\n\n---- Variable {var}")
        castle_md.compute_stats(i_time, var, nTime=n_time)
        stats_dict[str(var)] = castle_md.stats
                    
    f_name = f"hor_stats.p"
    with open(os.path.join(save_dir, f_name), "wb") as f:
        pickle.dump(stats_dict, f)
    

#### Time step range 

In [17]:
i_time = "range"
n_time = 1440

run_compute_stats()



---- Variable tphystnd-3.64

Computing horizontal stats for variable tphystnd-3.64

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-7.59

Computing horizontal stats for variable tphystnd-7.59

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-14.36

Computing horizontal stats for variable tphystnd-14.36

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-24.61

Computing horizontal stats for variable tphystnd-24.61

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-38.27

Computing horizontal stats for variable tphystnd-38.27

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-54.6

Computing horizontal stats for variable tphystnd-54.6

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-72.01

Computing horizontal stats for variable tphystnd-72.01

Test batch size = 8192.
Time samples: 1440


---- Variable tphystnd-87.82

Computing horizontal stats for variable tphystnd-87.82

T

divide by zero encountered in divide
divide by zero encountered in double_scalars


Time samples: 1440


---- Variable phq-14.36

Computing horizontal stats for variable phq-14.36

Test batch size = 8192.


divide by zero encountered in divide
divide by zero encountered in double_scalars


Time samples: 1440


---- Variable phq-24.61

Computing horizontal stats for variable phq-24.61

Test batch size = 8192.
Time samples: 1440


---- Variable phq-38.27

Computing horizontal stats for variable phq-38.27

Test batch size = 8192.
Time samples: 1440


---- Variable phq-54.6

Computing horizontal stats for variable phq-54.6

Test batch size = 8192.
Time samples: 1440


---- Variable phq-72.01

Computing horizontal stats for variable phq-72.01

Test batch size = 8192.
Time samples: 1440


---- Variable phq-87.82

Computing horizontal stats for variable phq-87.82

Test batch size = 8192.
Time samples: 1440


---- Variable phq-103.32

Computing horizontal stats for variable phq-103.32

Test batch size = 8192.
Time samples: 1440


---- Variable phq-121.55

Computing horizontal stats for variable phq-121.55

Test batch size = 8192.
Time samples: 1440


---- Variable phq-142.99

Computing horizontal stats for variable phq-142.99

Test batch size = 8192.
Time samples: 1440


---- Va

divide by zero encountered in divide


Time samples: 1440


---- Variable flnt

Computing horizontal stats for variable flnt

Test batch size = 8192.


divide by zero encountered in divide


Time samples: 1440


---- Variable flns

Computing horizontal stats for variable flns

Test batch size = 8192.
Time samples: 1440


---- Variable prect

Computing horizontal stats for variable prect

Test batch size = 8192.
Time samples: 1440


### Load and display

In [18]:
dict_file = Path(plot_dir, get_save_str(i_time, num_time=n_time), "hor_stats.p")
with open(dict_file, "rb") as f:
    stats_dict = pickle.load(f)

In [20]:
for key, items in stats_dict.items():
    print(f"\nStats for variable {key}:")
    for s, v in items.items():
        print(f"{s}: {v}")


Stats for variable tphystnd-3.64:
hor_tsqmean: 2.310132993799386e-09
hor_tmean: 3.025768035469561e-06
hor_mse: 2.797516563449432e-11
hor_tvar: 2.300977721594917e-09
hor_r2: 0.9878420528056641

Stats for variable tphystnd-7.59:
hor_tsqmean: 1.2246476527137705e-09
hor_tmean: 5.799085646043226e-07
hor_mse: 2.7885787535530107e-11
hor_tvar: 1.224311358770469e-09
hor_r2: 0.9772232877398648

Stats for variable tphystnd-14.36:
hor_tsqmean: 1.504981780182242e-09
hor_tmean: -1.072999353602802e-05
hor_mse: 6.554547937806651e-10
hor_tvar: 1.3898490188990388e-09
hor_r2: 0.5283985635361459

Stats for variable tphystnd-24.61:
hor_tsqmean: 1.1085736701045974e-09
hor_tmean: 2.0490010612237182e-05
hor_mse: 3.404378265043508e-10
hor_tvar: 6.887331352150051e-10
hor_r2: 0.50570430098724

Stats for variable tphystnd-38.27:
hor_tsqmean: 9.87209322563402e-11
hor_tmean: -1.082172218870208e-06
hor_mse: 1.4478147484467063e-11
hor_tvar: 9.754983554504574e-11
hor_r2: 0.8515820410811306

Stats for variable tphystn

In [ ]:
del castle_models

In [ ]:
del castle_md